# Install necessary packages

In [43]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Part 3: Practical Data Preparation

**Objective:** Handle categorical features using One-Hot Encoding and address class imbalance using SMOTE.

## 1. Setup

Import necessary libraries.

In [44]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

## 2. Data Loading

Load the dataset.

In [45]:
def load_data(file_path):
    """
    Load the synthetic health data from a CSV file.
    
    Args:
        file_path: Path to the CSV file
        
    Returns:
        DataFrame containing the data
    """
    # YOUR CODE HERE
    # Load the CSV file using pandas
    df = pd.read_csv(file_path)

    return df  # Replace with actual implementation

## 3. Categorical Feature Encoding

Implement `encode_categorical_features` using `OneHotEncoder`.

In [46]:
def encode_categorical_features(df, column_to_encode='smoker_status'):
    """
    Encode a categorical column using OneHotEncoder.
    
    Args:
        df: Input DataFrame
        column_to_encode: Name of the categorical column to encode
        
    Returns:
        DataFrame with the categorical column replaced by one-hot encoded columns
    """
    # YOUR CODE HERE
    # 1. Extract the column to encode
    categorical_values = df[[column_to_encode]]

    # 2. Apply OneHotEncoder
    encoder = OneHotEncoder(sparse_output = False, drop='first')
    encoded_array = encoder.fit_transform(categorical_values)

    # 3. Create new column names
    new_cols = encoder.get_feature_names_out([column_to_encode])
    encoded_df = pd.DataFrame(encoded_array, columns=new_cols, index=df.index)

    # 4. Replace the original column with new encoded columns
    df_encoded = df.drop(columns=[column_to_encode])
    df_encoded = pd.concat([df_encoded, encoded_df], axis=1)

    # Placeholder return - replace with your implementation
    return df_encoded

In [47]:
encode_categorical_features(df, column_to_encode='smoker_status')

,patient_id,timestamp,age,systolic_bp,diastolic_bp,glucose_level,bmi,heart_rate,disease_outcome,smoker_status_no,smoker_status_yes
0,1,2023-01-29 00:00:00.000000,57,113.063416,84.069561,117.475210,25.085796,62.719587,0,1.0,0.0
1,1,2023-01-31 07:33:55.507789,57,121.598849,89.672279,85.120875,24.120608,76.314434,0,1.0,0.0
2,1,2023-02-02 00:15:11.379377,57,126.623222,87.619685,NaN,24.819332,62.427785,0,1.0,0.0
3,1,2023-02-04 09:37:12.589164,57,136.999366,89.199774,118.755648,25.039598,61.612981,0,1.0,0.0
4,1,2023-02-04 20:56:52.838198,57,127.546919,92.644673,98.882007,24.895024,77.649615,0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
7321,150,2023-03-18 09:08:49.029823,54,115.038254,79.241741,84.586944,29.968156,73.599447,0,1.0,0.0
7322,150,2023-03-20 14:38:22.129593,54,116.389186,70.464818,91.476621,29.519510,64.162701,0,1.0,0.0
7323,150,2023-03-23 09:26:04.210673,54,123.419606,88.213054,96.985434,29.786678,71.641423,0,1.0,0.0
7324,150,2023-03-27 14:17:19.255961,54,NaN,69.539940,85.670800,29.188655,72.781243,0,1.0,0.0


## 4. Data Preparation

Implement `prepare_data_part3` to handle the train/test split correctly.

In [48]:
def prepare_data_part3(df, test_size=0.2, random_state=42):
    """
    Prepare data with categorical encoding.
    
    Args:
        df: Input DataFrame
        test_size: Proportion of data for testing
        random_state: Random seed for reproducibility
        
    Returns:
        X_train, X_test, y_train, y_test
    """
    # YOUR CODE HERE
    # 1. Encode categorical features using the encode_categorical_features function
    df_encoded = encode_categorical_features(df, column_to_encode='smoker_status')

    # 2. Select relevant features (including the one-hot encoded ones) and the target
    feature_cols = [
        'age', 'systolic_bp', 'diastolic_bp', 'glucose_level', 'bmi', 'heart_rate',
        'disease_outcome', 'smoker_status_no', 'smoker_status_yes'
    ]

    encoded_cols = [col for col in df_encoded.columns if col.startswith('smoker_status_')]
    features = feature_cols + encoded_cols

    X = df_encoded[features]
    y = df_encoded['disease_outcome']

    # 3. Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    # 4. Handle missing values
    imputer = SimpleImputer(strategy='mean')
    X_train = imputer.fit_transform(X_train)
    X_test = imputer.transform(X_test)

    # Placeholder return - replace with your implementation
    return X_train, X_test, y_train, y_test

## 5. Handling Imbalanced Data

Implement `apply_smote` to oversample the minority class.

In [49]:
def apply_smote(X_train, y_train, random_state=42):
    """
    Apply SMOTE to oversample the minority class.
    
    Args:
        X_train: Training features
        y_train: Training target
        random_state: Random seed for reproducibility
        
    Returns:
        Resampled X_train and y_train with balanced classes
    """
    # YOUR CODE HERE
    # Apply SMOTE to balance the classes
    smote = SMOTE(random_state=random_state)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
    
    # Placeholder return - replace with your implementation
    return X_resampled, y_resampled

## 6. Model Training and Evaluation

Train a model on the SMOTE-resampled data and evaluate it.

In [50]:
def train_logistic_regression(X_train, y_train):
    """
    Train a logistic regression model.
    
    Args:
        X_train: Training features
        y_train: Training target
        
    Returns:
        Trained logistic regression model
    """
    # YOUR CODE HERE
    # Initialize and train a LogisticRegression model
    model = LogisticRegression(max_iter=1000).fit(X_train, y_train)

    return model  # Replace with actual implementation

def calculate_evaluation_metrics(model, X_test, y_test):
    """
    Calculate classification evaluation metrics.
    
    Args:
        model: Trained model
        X_test: Test features
        y_test: Test target
        
    Returns:
        Dictionary containing accuracy, precision, recall, f1, auc, and confusion_matrix
    """
    # YOUR CODE HERE
    # 1. Generate predictions
    y_pred = model.predict(X_test)

    # 2. Calculate metrics: accuracy, precision, recall, f1, auc
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)

    # 3. Create confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # 4. Return metrics in a dictionary
    # Placeholder return - replace with your implementation
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        'confusion_matrix': cm
    }

## 7. Save Results

Save the evaluation metrics to a text file.

In [59]:
# YOUR CODE HERE
# 1. Create 'results' directory if it doesn't exist
# 2. Format metrics as strings
# 3. Write metrics to 'results/results_part3.txt'

def save_evaluation_metrics(metrics):
    # 1. Create 'results' directory if it doesn't exist
    os.makedirs('results', exist_ok=True)

    # 2. Format metrics as strings
    metrics_str = "# Logistic Regression Evaluation Metrics – Part 3\n\n"
    metrics_str += '\n'.join(
        f'{key}: {value:.4f}' if np.isscalar(value) else f'{key}:\n{np.array2string(value, precision=4)}'
        for key, value in metrics.items()
    )

    # 3. Write metrics to 'results/results_part3.txt'
    with open(os.path.join('results', 'results_part3.txt'), 'w') as f:
        f.write(metrics_str)


## 8. Main Execution

Run the complete workflow.

In [60]:
# Main execution
if __name__ == "__main__":
    # 1. Load data
    data_file = 'data/synthetic_health_data.csv'
    df = load_data(data_file)
    
    # 2. Prepare data with categorical encoding
    X_train, X_test, y_train, y_test = prepare_data_part3(df)
    
    # 3. Apply SMOTE to balance the training data
    X_train_resampled, y_train_resampled = apply_smote(X_train, y_train)
    
    # 4. Train model on resampled data
    model = train_logistic_regression(X_train_resampled, y_train_resampled)
    
    # 5. Evaluate on original test set
    metrics = calculate_evaluation_metrics(model, X_test, y_test)
    
    # 6. Print metrics
    for metric, value in metrics.items():
        if metric != 'confusion_matrix':
            print(f"{metric}: {value:.4f}")
    
    # 7. Save results
    # (Your code for saving results)
    save_evaluation_metrics(metrics)

    # 8. Load Part 1 results for comparison

    # NOTE: kept getting JSONDecodeError
    # import json
    # try:
    #     with open('results/results_part1.txt', 'r') as f:
    #         part1_metrics = json.load(f)
    # 8. Load Part 1 results for comparison
    try:
    # Read metrics from file directly without json
        part1_metrics = {}
        with open('results/results_part1.txt', 'r') as f:
            for line in f:
                if ':' in line:
                    key, value = line.strip().split(':')
                    key = key.strip().lower()
                    value = value.strip()  

                    if key == "confusion_matrix":
                        continue

                    else:
                        try:
                            value = float(value)
                        except ValueError:
                            pass

                    part1_metrics[key] = value

        # 9. Compare models
        comparison = compare_models(part1_metrics, metrics)
        print("\nModel Comparison (improvement percentages):")
        for metric, improvement in comparison.items():
            if improvement is None:
                print(f"{metric}: N/A")
            else:
                print(f"{metric}: {improvement:.2f}%")

    except FileNotFoundError:
        print("Part 1 results not found. Run part1_introduction.ipynb first.")

accuracy: 1.0000
precision: 1.0000
recall: 1.0000
f1_score: 1.0000
auc: 1.0000

Model Comparison (improvement percentages):
accuracy: 9.08%
precision: 51.17%
recall: 232.56%
f1_score: N/A
auc: 10.08%


## 9. Compare Results

Implement a function to compare model performance between balanced and imbalanced data.

In [53]:
def compare_models(part1_metrics, part3_metrics):
    """
    Calculate percentage improvement between models trained on imbalanced vs. balanced data.
    
    Args:
        part1_metrics: Dictionary containing evaluation metrics from Part 1 (imbalanced)
        part3_metrics: Dictionary containing evaluation metrics from Part 3 (balanced)
        
    Returns:
        Dictionary with metric names as keys and improvement percentages as values
    """
    # YOUR CODE HERE
    # 1. Calculate percentage improvement for each metric
    # 2. Handle metrics where higher is better (most metrics) and where lower is better
    # 3. Return a dictionary with metric names and improvement percentages
    
    improvements = {}
    
    # Define metrics to compare (skip confusion_matrix)
    for metric in ['accuracy', 'precision', 'recall', 'f1_score', 'auc']:
        val1 = part1_metrics.get(metric)
        val3 = part3_metrics.get(metric)
        
        if val1 is None or val3 is None:
            improvements[metric] = None
            continue

        if val1 == 0:
            improvements[metric] = float('inf')
        else:
            improvements[metric] = ((val3 - val1) / abs(val1)) * 100
    
    return improvements